In [1]:
import numpy as np
import pandas as pd
from scipy.optimize import fsolve
from Fixed_Income_Toolbox import *
import statsmodels.formula.api as sm

In [2]:
##### 1
maturity = np.arange(1,7)
spot = [0.05, 0.055, 0.057, 0.059, 0.06, 0.061]
price = []
for i in range(len(maturity)):
    bond = ZeroCouponBond(100, maturity[i])
    price.append(bond.get_price(spot[i]))
price

[95.23809523809524,
 89.84524157139327,
 84.6788669093383,
 79.50897588580243,
 74.7258172866057,
 70.09833403416522]

In [3]:
def ho_and_lee_calibrate(level, spot, vol, price, fv=100):
    calibrate_lv = [s + vol for s in spot[level - 1]] + [spot[level - 1][-1] - vol] # construct the last level of spot
    def function(x, *args):
        calibrate_lv, spot, price, fv = args
        price_tree = [fv / (1 + spot + x) for spot in calibrate_lv]
        for i in reversed(range(len(price_tree))):
            for j in range(i):
                price_tree[j] = (0.5 * price_tree[j] + 0.5 * price_tree[j+1]) / (1 + spot[i][j])
        return price_tree[0] - price
    args = (calibrate_lv, spot, price, fv)
    m = fsolve(function, 0.01, args=args)
    return [s + m[0] for s in calibrate_lv]

In [4]:
# construct the ho and lee tree
vol = 0.015
spot_dict = {1:[spot[0]]}
for lv in range(2, len(price)+1):
    spot_dict[lv] = ho_and_lee_calibrate(lv, spot_dict, vol, price[lv-1])

# output dict into tree
ho_lee_rates = pd.DataFrame()
def dict_to_tree(df,dic):    
    for i in range(1,len(dic)+1):
        for j in range(len(dic[i])):
            df.loc[j,i-1]=dic[i][j]
    return df           
dict_to_tree(ho_lee_rates,spot_dict)

,0,1,2,3,4,5
0,0.05,0.075236,0.091648,0.111292,0.126125,0.144184
1,NaN,0.045236,0.061648,0.081292,0.096125,0.114184
2,NaN,NaN,0.031648,0.051292,0.066125,0.084184
3,NaN,NaN,NaN,0.021292,0.036125,0.054184
4,NaN,NaN,NaN,NaN,0.006125,0.024184
5,NaN,NaN,NaN,NaN,NaN,-0.005816


In [5]:
###### 2
# a) Non-prepayable
def get_payoff_tree(spot_dict, cf_dict,T,k):
    sorted_lv = range(int(T*k),0,-1)
    last_period = sorted_lv[0]
    payoff_dict = {}
    for lv in sorted_lv:
        if lv == last_period:
            payoff_dict[lv] = np.divide(cf_dict[lv], [s + 1 for s in spot_dict[lv]]).tolist()
        else:
            payoff_dict[lv] = np.zeros(lv).tolist()
            for i in range(lv):
                payoff_dict[lv][i] = (0.5 * (payoff_dict[lv+1][i] + payoff_dict[lv+1][i+1]) + cf_dict[lv][i]) / (1 + spot_dict[lv][i])
    return payoff_dict

z_ls = [bond.get_discount_function(0.055, t) for t in range(1,7)]
fixed_pay = 100 / sum(z_ls) # coupon payment
fixed_cf = {i:[fixed_pay]*i for i in range(1, 7)}
fixed_payoff = get_payoff_tree(spot_dict, fixed_cf,6,1) 
# price
print(fixed_payoff[1][0])
# Duration
def tree_duration(price_dict,yield_dict):
    return -1/price_dict[1][0]*(price_dict[2][0]-price_dict[2][1])/(yield_dict[2][0]-yield_dict[2][1])
print(tree_duration(fixed_payoff, spot_dict))



98.90748336977826
2.2930854075484


In [6]:
# b) Prepayable
yield_dict = {i:[0.055]*i for i in range(1, 7)}
prepay_payoff = get_payoff_tree(yield_dict, fixed_cf,6,1) 
# Compare each CF node from fixed_payoff tree and prepay_payoff tree, take the lesser amount node and discount back 
def get_prepay_tree(regtree,preptree,spot_dict,cf_dict,func):
    sorted_lv = range(len(regtree),0,-1)
    last_period = sorted_lv[0]
    prep_node ={}
    payoff_dict={}
    temp_dict={}
    for lv in sorted_lv:
        if lv == last_period: 
            temp_dict[lv] = payoff_dict[lv] = [func(x,y) for x,y in zip(regtree[lv],preptree[lv])]
            prep_node[lv] = [int(x>y) for x,y in zip(regtree[lv],preptree[lv])]
        else:
            payoff_dict[lv] = prep_node[lv] = temp_dict[lv] = np.zeros(lv).tolist()
            for i in range(lv):
                temp_dict[lv][i] = (0.5 * (payoff_dict[lv+1][i] + payoff_dict[lv+1][i+1])+ cf_dict[lv][i])/ (1 + spot_dict[lv][i])
                payoff_dict[lv] = [func(x,y) for x,y in zip(temp_dict[lv],preptree[lv])]
                prep_node[lv] = [int(x>y) for x,y in zip(temp_dict[lv],preptree[lv])]
    return (payoff_dict, prep_node)

prepay_tree = get_prepay_tree(fixed_payoff,prepay_payoff,spot_dict,fixed_cf,min)[0]
prepay_node = get_prepay_tree(fixed_payoff,prepay_payoff,spot_dict,fixed_cf,min)[1]
print(prepay_tree[1][0])
print(tree_duration(prepay_tree,spot_dict))

    

98.01705337126697
1.7471496600989311


In [7]:
# c) PO Security
fixed_int = {i:[0]*i for i in range(1, 7)} # Non-prepayment Interest
fixed_po = {i:[0]*i for i in range(1, 7)} # Non-prepayment Principal
for lv in range(1,7):
    for i in range(lv):
        fixed_int[lv][i] = 0.055*prepay_payoff[lv][i]
        fixed_po[lv][i] = fixed_cf[lv][i] - fixed_int[lv][i]
        
# Prepayable PO cashflow tree 

def PO_cashflow(potree, balancetree, prepnode):
    po_cf = {i:[0]*i for i in range(1, len(potree)+1)}
    for lv in range(1,len(potree)+1):
        for i in range(lv):
            if prepnode[lv][i] == 1:
                po_cf[lv][i] = balancetree[lv][i]
            else:
                po_cf[lv][i] = potree[lv][i]
    return po_cf
   
po_cashflow = PO_cashflow(fixed_po, prepay_payoff, prepay_node) 

In [8]:
# Prepayable PO value tree
def PO_value(po_cf, prepnode, spot_dict):
    sorted_lv = range(len(po_cf),0,-1)
    last_period = sorted_lv[0]
    po_value = {i:[0]*i for i in range(1, len(po_cf)+1)}
    for lv in sorted_lv:
        for i in range(lv):
            if prepnode[lv][i] == 1:
                po_value[lv][i] = po_cf[lv][i]
            elif lv == last_period:
                po_value[lv][i] = po_cf[lv][i]/(1 + spot_dict[lv][i])
            else:
                po_value[lv][i] = (0.5 * (po_value[lv+1][i] + po_value[lv+1][i+1])+po_cf[lv][i]) / (1 + spot_dict[lv][i])
    return po_value
   
po_value = PO_value(po_cashflow, prepay_node, spot_dict) 
print(po_value[1][0]) # po price
print(tree_duration(po_value,spot_dict)) # po duration


83.19408135789386
3.4928422802975247


In [9]:
# d) IO Security
def IO_value(po_value, preptree):
    io_value = {i:[0]*i for i in range(1, len(po_value)+1)}
    for lv in range(1,len(po_value)+1):
        for i in range(lv):
            io_value[lv][i] = preptree[lv][i] - po_value[lv][i]
    return io_value

io_value = IO_value(po_value, prepay_tree)
print(io_value[1][0]) # IO price
print(tree_duration(io_value,spot_dict)) # IO duration
    

14.822972013373104
-8.050567946036384


In [13]:
# BDT calibration
def bdt_calibrate(level, spot, sigma, prop_vol, price, k=1, fv=100):
    def calibrate(m, level, spot, sigma):
        calibrate_lv = [spot[level-1][0] * np.exp(m + sigma[level-1])]
        for i in range(1, level):
            calibrate_lv += [calibrate_lv[i-1] * np.exp(- 2 * sigma[level-1])]
        return calibrate_lv
        
    def function(params, *args):
        level, spot, sigma, price, k, fv = args
        m, sigma[level-1] = params
        calibrate_lv = calibrate(m, level, spot, sigma)
        price_tree = [fv / (1 + spot / k) for spot in calibrate_lv]
        for i in reversed(range(len(price_tree))):
            for j in range(i):
                if i == 1:
                    bond = ZeroCouponBond(fv, level-1)
                    spot_u = bond.get_spot(price_tree[j], k=k)
                    spot_d = bond.get_spot(price_tree[j+1], k=k)
                price_tree[j] = (0.5 * price_tree[j] + 0.5 * price_tree[j+1]) / (1 + spot[i][j] / k)
        return [price_tree[0] - price, 0.5 * np.log(spot_u / spot_d) - prop_vol]
    
    args = (level, spot, sigma, price, k, fv)
    r = fsolve(function, [0.01, 0.1], args=args)
    return calibrate(r[0], level, spot, sigma)

In [11]:
# data from PS1
df = pd.read_excel('HW1_data.xls')
bond = ZeroCouponBond(100, df['Maturity'])
df['Spot'] = bond.get_spot(df['Price'], k=2)
# estimate term structure
new_df = pd.DataFrame(df['Maturity'].apply(lambda x: x**i) for i in range(1,6)).T
new_df.columns = ['M1','M2','M3','M4','M5']
df['Discount Function'] = bond.get_discount_function(df['Spot'], df['Maturity'], k=2)
new_df['logZ'] = np.log(df['Discount Function'])
rls = sm.ols(formula="logZ ~ %s + 0" % "+".join(new_df.loc[:,'M1':'M5'].columns.tolist()),data=new_df).fit()
# predict new maturity
predict_series = pd.Series(np.linspace(0.5, 10, 20))
predict_df = pd.DataFrame(predict_series.apply(lambda x: x**i) for i in range(1,6)).T
predict_df['Estimated Z'] = np.exp(np.dot(predict_df, rls.params))
predict_df['Price'] = 100 * predict_df['Estimated Z']
predict_df['Spot'] = (predict_df['Estimated Z']**(- 1 / (2 * predict_df.loc[:,0])) - 1) * 2
predict_df.head()

,0,1,2,3,4,Estimated Z,Price,Spot
0,0.5,0.25,0.125,0.0625,0.03125,0.983559,98.355909,0.033431
1,1.0,1.00,1.000,1.0000,1.00000,0.966855,96.685521,0.033992
2,1.5,2.25,3.375,5.0625,7.59375,0.949903,94.990280,0.034559
3,2.0,4.00,8.000,16.0000,32.00000,0.932721,93.272057,0.035130
4,2.5,6.25,15.625,39.0625,97.65625,0.915331,91.533093,0.035703


In [14]:
# calibrate 10 years model
prop_vol = 0.15 * np.sqrt(0.5)
spot_dict = {1:[predict_df.loc[0, 'Spot']]}
sigma = {1:prop_vol}
for lv in range(2, len(predict_df.index)+1):
    spot_dict[lv] = bdt_calibrate(lv, spot_dict, sigma, prop_vol, predict_df.loc[lv-1, 'Price'], k=2)
spot_dict

{1: [0.033431460302576266],
 2: [0.03822701005487272, 0.030892244589409704],
 3: [0.043698678889266075, 0.03531151825157295, 0.02853411940417843],
 4: [0.049933976824511665,
  0.040345359359113375,
  0.03259800491229783,
  0.026338343272735334],
 5: [0.057031985063596954,
  0.04607277246779216,
  0.037219471854291974,
  0.03006741315775687,
  0.024289687331901254],
 6: [0.06510518733327798,
  0.05258327027323487,
  0.042469738985218435,
  0.034301379889463884,
  0.027704070955812253,
  0.02237564640250617],
 7: [0.07428165153039433,
  0.059978556373546064,
  0.04842955360493916,
  0.039104336686038334,
  0.03157471076708605,
  0.02549493085714689,
  0.020585825922695748],
 8: [0.0847076346282119,
  0.06837453156829666,
  0.055190734432654806,
  0.04454900234560701,
  0.03595918101815762,
  0.02902562642065848,
  0.023428981563462556,
  0.018911467030747222],
 9: [0.09655069999289533,
  0.07790369125620596,
  0.06285801254458896,
  0.050718132572966486,
  0.0409228492527392,
  0.0330193

In [ ]:
bdt_dict = {i:[0]*i for i in range(1, 21)} 
bdt_dict2 = {i:[0]*i for i in range(1, 21)} 
for lv in range(1,len(spot_dict)+1):
    for i in range(lv):
        bdt_dict[lv][i] = spot_dict[lv][i] * 0.01*.5
        bdt_dict2[lv][i] = spot_dict[lv][i] * 0.01



In [ ]:
##### 3
# b) Callable bond
def generate_cf(T,coupon,k=2,fv=100):
    cf = fv*coupon/k
    cf0 = {i:[cf]*i for i in range(1, int(k*T))}
    cf1 = {i:[cf+fv]*i for i in range(int(k*T), int(k*T+1))}
    cf0.update(cf1)
    return cf0
bond_cf = generate_cf(10,.04)

# Noncallable bond payoff
nc_bond = get_payoff_tree(bdt_dict, bond_cf,10,2) 
# American call payoff
def american_call(nc_bond, spot_dict, strike, t, k): #t is when the call starts
    sorted_lv = range(len(nc_bond),0,-1)
    last_period = sorted_lv[0]
    call_value = temp_value = {i:[0]*i for i in range(1, len(nc_bond)+1)}
    for lv in sorted_lv:
        for i in range(lv):
            if lv > (t*k):
                if lv == last_period:
                    call_value[lv][i] = temp_value[lv][i] = max(nc_bond[lv][i] - strike, 0)
                else:
                    temp_value[lv][i] = (0.5 * (call_value[lv+1][i] + call_value[lv+1][i+1])) / (1 + spot_dict[lv][i])
                    call_value[lv][i] = max(nc_bond[lv][i]- strike, temp_value[lv][i])
            else:
                call_value[lv][i] = (0.5 * (call_value[lv+1][i] + call_value[lv+1][i+1])) / (1 + spot_dict[lv][i])
    return call_value


def callable_bond(nc_bond, call):
    cbond = {i:[0]*i for i in range(1, len(nc_bond)+1)}
    for lv in range(1,len(nc_bond)):
        cbond[lv] = np.subtract(nc_bond[lv], call[lv])
    return cbond

call = american_call(nc_bond, bdt_dict, 100, 2, 2)                    
cbond = callable_bond(nc_bond,call)
cbond[1][0]  

In [ ]:
# c) Foward 
def get_tree_discount (spot_dict, T, k, fv=100):
    cf = generate_cf(T,coupon=0,k=k,fv=fv)
    spot = {k: spot_dict[k] for k in range(1,int(T*k)+1)}
    z = get_payoff_tree(spot, cf,T,k)
    return z

z1 = get_tree_discount(bdt_dict, 1, 2)
z10 = get_tree_discount(bdt_dict, 10, 2)
z1_price = z1[1][0]
z10_price = z10[1][0]
foward = z10_price/z1_price*100
foward


In [ ]:
# d) Future
def future_price(bond_price, spot_dict,T,k): # T is maturity of future
    sorted_lv = range(int(k*T),0,-1)
    last_period = sorted_lv[0]
    future_val = {i:[0]*i for i in range(1, int(k*T)+1)}
    for lv in sorted_lv:
        if lv == last_period:
            future_val[lv] = bond_price[lv]
        else:
            for i in range(lv):
                future_val[lv][i] = 0.5 * (future_val[lv+1][i] + future_val[lv+1][i+1])
    return future_val
            
future = future_price(z10,bdt_dict,1,2) 
future[1][0]
     

In [ ]:
# e) Hedge with forward
def delta(price_dict,yield_dict,pv):
    delta = - tree_duration(price_dict,yield_dict)*pv
    return delta
    
D_cbond = delta(cbond,bdt_dict2,cbond[1][0])
D_z1 = delta(z1,bdt_dict2,z1_price)
D_z10 = delta(z10,bdt_dict2,z10_price)
D_forward = D_z10 - D_z1
pos_forward = -D_cbond/D_forward
pos_forward

In [ ]:
# f) Hedge with future
D_future = delta(future,bdt_dict2,future[1][0])
pos_future = - D_cbond/D_future
pos_future

In [ ]:
# g) Swaption 
def swap_rate(spot_dict,T,k,fv=1):
    z_ls = [get_tree_discount(spot_dict, t, k, fv)[1][0] for t in np.arange(1,T+0.5,0.5)]
    c = k*(1-get_tree_discount(spot_dict, T, k, fv)[1][0])/sum(z_ls)
    return c

swap_r = swap_rate(bdt_dict,10,2)
swap_r


In [ ]:
def swap_cashflow(spot_dict,swap_rate,T,k,fv=100,type = "receiver"):
    sorted_lv = range(int(k*T),0,-1)
    cf = {i:[0]*i for i in range(1, len(sorted_lv)+1)}
    for lv in sorted_lv:
        for i in range(lv):
            if type == "receiver":
                cf[lv][i] = fv*(1/k)*(swap_rate - spot_dict[lv][i])
            else:
                cf[lv][i] = fv*(1/k)*(spot_dict[lv][i] - swap_rate)
    return cf

swap_cf = swap_cashflow(bdt_dict,swap_r,5,2)    
swap_val = get_payoff_tree(bdt_dict, swap_cf,5,2)

In [ ]:
# European swaption value
euro_swaption_cf = generate_cf(4.5,coupon=0,k=2,fv=0)
l = {10:[max(k,0) for k in swap_val[10]]} # European swaption final payoff
euro_swaption_cf.update(l)
euro_swaption = get_payoff_tree(bdt_dict, euro_swaption_cf,5,2)

In [ ]:
def swap_american(swap_euro,spot_dict,swap_rate,T,k,fv=100):
    sorted_lv = range(int(k*T),0,-1)
    last_period = sorted_lv[0]
    val = {i:[0]*i for i in range(1, len(sorted_lv)+1)}
    temp = {i:[0]*i for i in range(1, len(sorted_lv)+1)}
    for lv in sorted_lv:
        cf = swap_cashflow(spot_dict,swap_rate,lv/k,k,fv)
        exercise = get_payoff_tree(spot_dict,cf,lv/k,k)[lv]
        for i in range(lv):
            if lv == last_period: 
                val[lv][i] = temp[lv][i] = max(0,exercise[i])
            else:
                temp[lv][i] = (0.5 * (val[lv+1][i] + val[lv+1][i+1])) / (1 + spot_dict[lv][i])
                val[lv][i] = max(temp[lv][i],exercise[i])
    return val        
    
swap_american(euro_swaption,bdt_dict,swap_r,5,2)    

In [ ]:
euro_swaption